In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests 
df = pd.read_csv('../data/raw/ems_2025_merged.csv')
df

,incident_id,datetime,initial_type,initial_severity,final_type,final_severity,assignment_time,valid_dispatch_response,dispatch_response_sec,activation_time,...,datetime_hour,datetime_weather,temperture,precipitation,windspeed,weathercode,latitude_of_call,longitude_of_call,closest_station_name,closest_station_manhattan_miles
0,250010001,2025-01-01 00:00:12,STNDBY,8,STNDBY,8,2025-01-01T09:53:41.000,N,0,2025-01-01T09:57:09.000,...,2025-01-01 00:00:00,2025-01-01 00:00:00,46.8,0.0,10.7,3,40.5771,-73.9881,Engine 318/Ladder 166,0.300912
1,250010003,2025-01-01 00:01:53,UNC,2,UNC,2,2025-01-01T00:02:06.000,Y,13,2025-01-01T00:02:43.000,...,2025-01-01 00:00:00,2025-01-01 00:00:00,46.8,0.0,10.7,3,40.7506,-73.9971,Engine 34/Ladder 21,0.446225
2,250010004,2025-01-01 00:01:58,CARD,3,CARD,3,2025-01-01T00:01:58.000,Y,0,2025-01-01T00:01:58.000,...,2025-01-01 00:00:00,2025-01-01 00:00:00,46.8,0.0,10.7,3,40.7590,-73.9899,Battalion 9/Engine 54/Ladder 4,0.280244
3,250010007,2025-01-01 00:03:42,ABDPN,5,ABDPN,5,2025-01-01T00:03:59.000,Y,17,2025-01-01T00:04:12.000,...,2025-01-01 00:00:00,2025-01-01 00:00:00,46.8,0.0,10.7,3,40.8583,-73.9301,Engine 95/Ladder 36,0.712385
4,250010008,2025-01-01 00:04:36,STATEP,2,STATEP,2,2025-01-01T00:05:06.000,Y,30,2025-01-01T00:05:18.000,...,2025-01-01 00:00:00,2025-01-01 00:00:00,46.8,0.0,10.7,3,40.8899,-73.8466,Battalion 15/Engine 63/Ladder 39,0.668752
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1079486,252435374,2025-08-31 23:58:45,UNKNOW,4,UNKNOW,4,2025-09-01T00:41:26.000,Y,2561,2025-09-01T00:42:18.000,...,2025-08-31 23:00:00,2025-08-31 23:00:00,73.8,0.0,9.2,3,40.8263,-73.8926,Engine 82/Ladder 31,0.292276
1079487,252435375,2025-08-31 23:58:59,OTHER,6,OTHER,6,2025-08-31T23:59:08.000,Y,9,2025-08-31T23:59:21.000,...,2025-08-31 23:00:00,2025-08-31 23:00:00,73.8,0.0,9.2,3,40.6013,-73.7565,Engine 264/Engine 328/Ladder 134,0.470051
1079488,252435378,2025-08-31 23:59:22,INJURY,5,INJURY,5,2025-08-31T23:59:37.000,Y,15,2025-08-31T23:59:57.000,...,2025-08-31 23:00:00,2025-08-31 23:00:00,73.8,0.0,9.2,3,40.6277,-73.9466,Engine 255/Ladder 157,0.813416
1079489,252435381,2025-08-31 23:59:52,MVAINJ,5,MVAINJ,5,2025-08-31T23:59:52.000,Y,0,2025-08-31T23:59:52.000,...,2025-08-31 23:00:00,2025-08-31 23:00:00,73.8,0.0,9.2,3,40.7127,-73.9530,Engine 221/Ladder 104,0.454651


In [2]:
# Random Forest averages all the tree(response) predictions to get the final result (time).
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [3]:
df.columns

Index(['incident_id', 'datetime', 'initial_type', 'initial_severity',
       'final_type', 'final_severity', 'assignment_time',
       'valid_dispatch_response', 'dispatch_response_sec', 'activation_time',
       'on_scene_time', 'valid_incident_response', 'travel_time_sec',
       'close_time', 'held', 'disposition', 'borough', 'dispatch_area',
       'zipcode', 'precinct', 'council_district', 'community_district',
       'school_district', 'congressional_district', 'reopened',
       'special_events', 'standby', 'transferred', 'incident_response_sec',
       'to_hospital_time', 'hospital_arrival_time', 'year', 'month', 'day',
       'week_day', 'hour', 'is_weekend', 'is_rush_hour', 'time_of_day',
       'season', 'is_holiday', 'datetime_hour', 'datetime_weather',
       'temperture', 'precipitation', 'windspeed', 'weathercode',
       'latitude_of_call', 'longitude_of_call', 'closest_station_name',
       'closest_station_manhattan_miles'],
      dtype='object')

In [28]:
# Winter Model
df_winter = df[df['month'].isin([1, 2, 3])].copy()
df_winter = df_winter[df_winter['incident_response_sec'].notna()]
df_winter = df_winter[df_winter['incident_response_sec'] < 7200]

X_winter = df_winter[['closest_station_manhattan_miles','hour','is_rush_hour','initial_severity','borough','temperture','precipitation','windspeed','weathercode']]
y_winter = df_winter['incident_response_sec']

X_winter_enc = pd.get_dummies(X_winter, columns=['borough','is_rush_hour'], drop_first=True)


X_test = X_test.reindex(columns=X_train.columns, fill_value=0)
X_train, X_test, y_train, y_test = train_test_split(X_winter_enc, y_winter, test_size=0.2, random_state=42)


winter_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
winter_model.fit(X_train, y_train)


,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsample

In [25]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [31]:
winter_pred = winter_model.predict(X_test)
print(f"Winter Model RMSE: {np.sqrt(mean_squared_error(y_test, winter_pred)):.2f} seconds")
winter_avg = df[df['month'].isin([1,2,3])]['incident_response_sec'].mean()
print(f"Winter (Jan-Mar): {winter_avg/60:.2f} minutes")

Winter Model RMSE: 718.58 seconds
Winter (Jan-Mar): 14.78 minutes


In [ ]:
# Spring Model
df_spring = df[df['month'].isin([4, 5, 6])].copy()
df_spring = df_spring[df_spring['incident_response_sec'].notna()] # since so much data i just used what was na
df_spring = df_spring[df_spring['incident_response_sec'] < 7200]

X_spring = df_spring[['closest_station_manhattan_miles','hour','is_rush_hour','initial_severity','borough','temperture','precipitation','windspeed','weathercode']]
y_spring = df_spring['incident_response_sec']

X_spring_enc = pd.get_dummies(X_spring, columns=['borough','is_rush_hour'], drop_first=True)

X_test = X_test.reindex(columns=X_train.columns, fill_value=0)
X_train, X_test, y_train, y_test = train_test_split(X_spring_enc, y_spring, test_size=0.2, random_state=42)


spring_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
spring_model.fit(X_train, y_train)

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsample

In [35]:
spring_pred = spring_model.predict(X_test)
print(f"Spring Model RMSE: {np.sqrt(mean_squared_error(y_test, spring_pred)):.2f} seconds")
spring_avg = df[df['month'].isin([4, 5, 6])]['incident_response_sec'].mean()
print(f"Spring (Apr-Jun): {spring_avg/60:.2f} minutes")

Spring Model RMSE: 734.57 seconds
Spring (Apr-Jun): 14.91 minutes


In [17]:
# Summer model
df_summer = df[df['month'].isin([7,8,9])].copy()
df_summer = df_summer[df_summer['incident_response_sec'].notna()]
df_summer = df_summer[df_summer['incident_response_sec']< 7200]

X_summer = df_summer[['closest_station_manhattan_miles','hour','is_rush_hour','initial_severity','borough','temperture','precipitation','windspeed','weathercode']]
y_summer = df_summer['incident_response_sec']

X_summer_enc = pd.get_dummies(X_summer, columns=['borough', 'is_rush_hour'], drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X_summer_enc,y_summer, random_state=42, test_size=0.2)

summer_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
summer_model.fit(X_train,y_train)


,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsample

In [ ]:
summer_pred = summer_model.predict(X_test)
print(f" Summer Model RMSE: {np.sqrt(mean_squared_error(y_test, summer_pred)):.2f} seconds")
summer_avg = df[df['month'].isin([7,8, 9])]['incident_response_sec'].mean()
print(f" Summer (Jul-Sep): {summer_avg/60:.2f} minutes")

Summer Model RMSE: 812.07 seconds
Spring (Jul-Sep): 15.81 minutes


In [ ]:
# first try 17 min is bad (bad date set)
# second try is 11 minutes lowered the time less than 2 hours (used less than 2 hours)
# third try with new data set 13 min
# fourth try 12 min using by season winter
# once broken down by season i got better results 15 min avg response time
# with 3 seasons is it still posibile to show case this 
# possible hypothesis testing 
#

In [23]:
overall_avg = df['incident_response_sec'].mean()
print(f"\nOverall Average: {overall_avg/60:.2f} minutes")


Overall Average: 15.09 minutes
